Based on the baseline "Radek's LGBMRanker starter-pack”: https://www.kaggle.com/code/marcogorelli/radek-s-lgbmranker-starter-pack


# Helper functions

In [ ]:
import numpy as np
import numpy as np
from tqdm import tqdm
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.

    This function computes the average prescision at k between two lists of
    items.

    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The average precision at k over the input lists

    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.

    This function computes the mean average prescision at k between two lists
    of lists of items.

    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The mean average precision at k over the input lists

    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

# https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations/discussion/308635
def customer_hex_id_to_int(series):
    return series.str[-16:].apply(hex_id_to_int)

def hex_id_to_int(str):
    return int(str[-16:], 16)

def article_id_str_to_int(series):
    return series.astype('int32')

def article_id_int_to_str(series):
    return '0' + series.astype('str')

class Categorize(BaseEstimator, TransformerMixin):
    def __init__(self, min_examples=0):
        self.min_examples = min_examples
        self.categories = []

    def fit(self, X):
        for i in range(X.shape[1]):
            vc = X.iloc[:, i].value_counts()
            self.categories.append(vc[vc > self.min_examples].index.tolist())
        return self

    def transform(self, X):
        data = {X.columns[i]: pd.Categorical(X.iloc[:, i], categories=self.categories[i]).codes for i in range(X.shape[1])}
        return pd.DataFrame(data=data)


def calculate_apk(list_of_preds, list_of_gts):
    # for fast validation this can be changed to operate on dicts of {'cust_id_int': [art_id_int, ...]}
    # using 'data/val_week_purchases_by_cust.pkl'
    apks = []
    for preds, gt in zip(list_of_preds, list_of_gts):
        apks.append(apk(gt, preds, k=12))
    return np.mean(apks)

def eval_sub(sub_csv, skip_cust_with_no_purchases=True):
    sub=pd.read_csv(sub_csv)
    validation_set=pd.read_parquet('data/validation_ground_truth.parquet')

    apks = []

    no_purchases_pattern = []
    for pred, gt in zip(sub.prediction.str.split(), validation_set.prediction.str.split()):
        if skip_cust_with_no_purchases and (gt == no_purchases_pattern): continue
        apks.append(apk(gt, pred, k=12))
    return np.mean(apks)

In [ ]:
import pandas as pd

# Read input files

In [ ]:
%%time
pad = "/kaggle/input/makeparquet"
transactions = pd.read_parquet(pad+'/transactions_train.parquet')
customers = pd.read_parquet(pad+'/customers.parquet')
articles = pd.read_parquet(pad+'/articles.parquet')

In [ ]:
test_week = transactions.week.max() + 1
transactions = transactions[transactions.week > transactions.week.max() - 10]

# Generating candidates

### Last purchase candidates

In [ ]:
%%time

c2weeks = transactions.groupby('customer_id')['week'].unique()

In [ ]:
transactions.groupby('week')['t_dat'].agg(['min', 'max'])

In [ ]:
%%time

c2weeks2shifted_weeks = {}

for c_id, weeks in c2weeks.items():
    c2weeks2shifted_weeks[c_id] = {}
    for i in range(weeks.shape[0]-1):
        c2weeks2shifted_weeks[c_id][weeks[i]] = weeks[i+1]
    c2weeks2shifted_weeks[c_id][weeks[-1]] = test_week

In [ ]:
candidates_last_purchase = transactions.copy()

In [ ]:
%%time

weeks = []
for i, (c_id, week) in enumerate(zip(transactions['customer_id'], transactions['week'])):
    weeks.append(c2weeks2shifted_weeks[c_id][week])

candidates_last_purchase.week=weeks

### Bestsellers candidates

In [ ]:
mean_price = transactions \
    .groupby(['week', 'article_id'])['price'].mean()

In [ ]:
sales = transactions \
    .groupby('week')['article_id'].value_counts() \
    .groupby('week').rank(method='dense', ascending=False) \
    .groupby('week').head(24).rename('bestseller_rank').astype('int8')

In [ ]:
bestsellers_previous_week2 = pd.merge(sales, mean_price, on=['week', 'article_id']).reset_index()
bestsellers_previous_week2.week += 1

In [ ]:
bestsellers_previous_week2.pipe(lambda df: df[df['week']==96])

In [ ]:
bestsellers_previous_week2

# Combining transactions and candidates / negative examples

### Add bestseller information

# Model training

In [ ]:
from lightgbm.sklearn import LGBMRanker

# Calculate predictions

In [ ]:
%time

bestsellers_last_week = \
    bestsellers_previous_week2[bestsellers_previous_week2.week == bestsellers_previous_week2.week.max()]['article_id'].tolist()

# Create submission

In [ ]:
sub = pd.read_csv('/kaggle/input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')

In [ ]:
latest_bought_articles = pd.read_csv('/kaggle/input/workingpurchase/filename.csv')

In [ ]:
latest_bought_articles = latest_bought_articles.values.tolist()


# Change repurchase to similar items with same group code

In [ ]:
# Function to replace article_id with random similar articles
def replace_article_ids(article_ids):
    new_article_ids = []

    for article_id in (article_ids):
        if article_id !=  0:
            product_code = articles.loc[articles['article_id'] == article_id, 'product_code'].values[0]
            similar_articles = articles.loc[articles['product_code'] == product_code, 'article_id'].tolist()

            # Exclude the original article_id from similar articles
            similar_articles = [a for a in similar_articles if a not in article_ids]

            # Shuffle the list of similar articles

            # Take up to 12 random similar articles
            new_article_ids.extend(similar_articles)
    np.random.shuffle(new_article_ids)
    new_article_ids[:min(12, len(new_article_ids))]
    while len(new_article_ids)<12:
        new_article_ids.append(0)
    return new_article_ids
new_last_bought_articles = []
# Replace article_ids in the last_bought_articles list
new_last_bought_articles = [replace_article_ids(article_ids) for article_ids in tqdm(latest_bought_articles)]



In [ ]:
print(new_last_bought_articles[:3])

In [ ]:
preds = [' '.join(['0' + str(p) for p in ps]) for ps in new_last_bought_articles]
sub.prediction = preds

In [ ]:
sub_name = 'similarity_submission'
# sub_name = 'popularity_submission'
sub.to_csv(f'{sub_name}.csv.gz', index=False)